In [1]:
import numpy as np
from PIL import Image, ImageTk, ImageDraw
import tkinter as tk
from tkinter import filedialog, messagebox, colorchooser
import os
import matplotlib.pyplot as plt

# Function to preprocess input image
def preprocess_image(image_path, size=(512, 512)):
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image = image.resize(size, Image.BICUBIC)
    return image

# Function to apply colors to specific regions
def apply_color_to_regions(grayscale_image, regions):
    # Convert grayscale to RGB
    rgb_image = grayscale_image.convert('RGB')
    rgb_array = np.array(rgb_image)
    
    # Create a copy for the result
    result_array = rgb_array.copy()
    
    for region, color in regions:
        x1, y1, x2, y2 = region
        # Ensure coordinates are within image bounds
        x1, y1 = max(0, int(x1)), max(0, int(y1))
        x2, y2 = min(rgb_array.shape[1], int(x2)), min(rgb_array.shape[0], int(y2))
        
        if x2 > x1 and y2 > y1:
            # Get the grayscale intensity for the region
            gray_values = rgb_array[y1:y2, x1:x2, 0]  # All channels are same in grayscale
            
            # Normalize grayscale values to [0, 1]
            gray_normalized = gray_values / 255.0
            
            # Apply color while preserving brightness/contrast
            for c in range(3):  # RGB channels
                colored_channel = gray_normalized * (color[c] / 255.0) * 255
                result_array[y1:y2, x1:x2, c] = colored_channel.astype(np.uint8)
    
    return Image.fromarray(result_array)

# Function to blend colors smoothly (optional enhancement)
def apply_color_with_blending(grayscale_image, regions, blend_strength=0.7):
    # Convert grayscale to RGB
    rgb_image = grayscale_image.convert('RGB')
    rgb_array = np.array(rgb_image, dtype=np.float32)
    result_array = rgb_array.copy()
    
    for region, color in regions:
        x1, y1, x2, y2 = region
        x1, y1 = max(0, int(x1)), max(0, int(y1))
        x2, y2 = min(rgb_array.shape[1], int(x2)), min(rgb_array.shape[0], int(y2))
        
        if x2 > x1 and y2 > y1:
            # Get grayscale values for the region
            gray_values = rgb_array[y1:y2, x1:x2, 0]
            
            # Create colored version
            colored_region = np.zeros((y2-y1, x2-x1, 3))
            for c in range(3):
                colored_region[:, :, c] = gray_values * (color[c] / 255.0)
            
            # Blend original grayscale with colored version
            original_region = rgb_array[y1:y2, x1:x2]
            result_array[y1:y2, x1:x2] = (
                original_region * (1 - blend_strength) + 
                colored_region * blend_strength
            )
    
    return Image.fromarray(result_array.astype(np.uint8))

# GUI class for manual colorization
class ManualColorizationGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Manual Conditional Image Colorization")
        self.regions = []
        self.start_x = None
        self.start_y = None
        self.rect = None
        self.original_image = None
        self.current_color = (255, 0, 0)  # Default red color
        self.blend_mode = True  # Toggle for blending

        # GUI elements
        self.label = tk.Label(root, text="Upload a grayscale image:")
        self.label.pack(pady=10)

        self.upload_btn = tk.Button(root, text="Upload Image", command=self.upload_image, 
                                   bg='lightblue', font=('Arial', 12))
        self.upload_btn.pack(pady=5)

        # Canvas frame
        self.canvas_frame = tk.Frame(root)
        self.canvas_frame.pack(pady=10)

        tk.Label(self.canvas_frame, text="Original", font=('Arial', 10, 'bold')).grid(row=0, column=0)
        tk.Label(self.canvas_frame, text="Colorized", font=('Arial', 10, 'bold')).grid(row=0, column=1)

        self.canvas_input = tk.Canvas(self.canvas_frame, width=400, height=400, bg='white', relief='sunken', bd=2)
        self.canvas_input.grid(row=1, column=0, padx=10)
        
        self.canvas_output = tk.Canvas(self.canvas_frame, width=400, height=400, bg='white', relief='sunken', bd=2)
        self.canvas_output.grid(row=1, column=1, padx=10)

        # Bind mouse events for region selection
        self.canvas_input.bind("<Button-1>", self.start_rectangle)
        self.canvas_input.bind("<B1-Motion>", self.update_rectangle)
        self.canvas_input.bind("<ButtonRelease-1>", self.end_rectangle)

        # Control buttons frame
        control_frame = tk.Frame(root)
        control_frame.pack(pady=10)

        self.color_btn = tk.Button(control_frame, text="Choose Color", command=self.choose_color,
                                  bg='lightgreen', font=('Arial', 11))
        self.color_btn.pack(side=tk.LEFT, padx=5)

        self.clear_btn = tk.Button(control_frame, text="Clear Regions", command=self.clear_regions,
                                  bg='orange', font=('Arial', 11))
        self.clear_btn.pack(side=tk.LEFT, padx=5)

        self.blend_btn = tk.Button(control_frame, text="Toggle Blend Mode", command=self.toggle_blend,
                                  bg='yellow', font=('Arial', 11))
        self.blend_btn.pack(side=tk.LEFT, padx=5)

        self.process_btn = tk.Button(control_frame, text="Apply Colors", command=self.process_image,
                                    bg='lightcoral', font=('Arial', 12, 'bold'))
        self.process_btn.pack(side=tk.LEFT, padx=5)

        self.save_btn = tk.Button(control_frame, text="Save Image", command=self.save_image,
                                 bg='lightsteelblue', font=('Arial', 11))
        self.save_btn.pack(side=tk.LEFT, padx=5)

        # Status frame
        status_frame = tk.Frame(root)
        status_frame.pack(pady=5)

        self.status_label = tk.Label(status_frame, text="Ready", font=('Arial', 10))
        self.status_label.pack()

        self.image_path = None
        self.output_image = None
        self.display_image = None

    def upload_image(self):
        self.image_path = filedialog.askopenfilename(
            title="Select Grayscale Image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.tiff")]
        )
        if self.image_path:
            self.original_image = preprocess_image(self.image_path)
            self.display_original()
            self.clear_regions()
            self.status_label.config(text=f"Loaded: {os.path.basename(self.image_path)}")

    def display_original(self):
        if self.original_image:
            # Resize for display
            display_img = self.original_image.resize((400, 400), Image.BICUBIC)
            self.display_image = ImageTk.PhotoImage(display_img)
            self.canvas_input.delete("all")
            self.canvas_input.create_image(200, 200, image=self.display_image)

    def start_rectangle(self, event):
        if not self.original_image:
            messagebox.showwarning("Warning", "Please upload an image first!")
            return
            
        self.start_x = event.x
        self.start_y = event.y
        if self.rect:
            self.canvas_input.delete(self.rect)
        self.rect = self.canvas_input.create_rectangle(
            self.start_x, self.start_y, self.start_x, self.start_y,
            outline='red', width=2, tags="current_rect"
        )

    def update_rectangle(self, event):
        if self.start_x is not None and self.start_y is not None:
            self.canvas_input.coords(self.rect, self.start_x, self.start_y, event.x, event.y)

    def end_rectangle(self, event):
        if self.start_x is not None and self.start_y is not None:
            end_x, end_y = event.x, event.y
            
            # Calculate actual coordinates (canvas is 400x400, image is 512x512)
            scale_x = 512 / 400
            scale_y = 512 / 400
            
            x1 = int(min(self.start_x, end_x) * scale_x)
            y1 = int(min(self.start_y, end_y) * scale_y)
            x2 = int(max(self.start_x, end_x) * scale_x)
            y2 = int(max(self.start_y, end_y) * scale_y)
            
            # Only add region if it has reasonable size
            if abs(x2 - x1) > 10 and abs(y2 - y1) > 10:
                self.regions.append(((x1, y1, x2, y2), self.current_color))
                # Change rectangle color to show it's been added
                self.canvas_input.itemconfig(self.rect, outline='blue', width=1)
                self.status_label.config(text=f"Added region {len(self.regions)} with color {self.current_color}")
            else:
                self.canvas_input.delete(self.rect)
                
            self.start_x = None
            self.start_y = None
            self.rect = None

    def choose_color(self):
        color = colorchooser.askcolor(title="Choose Color for Selected Region")
        if color[0]:  # color[0] is RGB tuple, color[1] is hex
            self.current_color = tuple(int(c) for c in color[0])
            self.color_btn.config(bg=color[1])
            self.status_label.config(text=f"Selected color: RGB{self.current_color}")

    def clear_regions(self):
        self.regions = []
        self.canvas_input.delete("current_rect")
        self.canvas_output.delete("all")
        self.status_label.config(text="Cleared all regions")

    def toggle_blend(self):
        self.blend_mode = not self.blend_mode
        mode_text = "Blend ON" if self.blend_mode else "Blend OFF"
        self.blend_btn.config(text=f"Blend Mode: {mode_text}")
        self.status_label.config(text=f"Blend mode: {mode_text}")

    def process_image(self):
        if not self.original_image:
            messagebox.showerror("Error", "Please upload an image first!")
            return
            
        if not self.regions:
            messagebox.showwarning("Warning", "Please select at least one region to colorize!")
            return

        try:
            # Apply colorization
            if self.blend_mode:
                self.output_image = apply_color_with_blending(self.original_image, self.regions)
            else:
                self.output_image = apply_color_to_regions(self.original_image, self.regions)
            
            # Display result
            display_output = self.output_image.resize((400, 400), Image.BICUBIC)
            self.output_photo = ImageTk.PhotoImage(display_output)
            self.canvas_output.delete("all")
            self.canvas_output.create_image(200, 200, image=self.output_photo)
            
            self.status_label.config(text=f"Applied colors to {len(self.regions)} regions")
            
            # Save comparison visualization
            self.save_visualization()
            
        except Exception as e:
            messagebox.showerror("Error", f"Colorization failed: {str(e)}")

    def save_visualization(self):
        try:
            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(self.original_image, cmap='gray')
            plt.title('Original Grayscale', fontsize=14)
            plt.axis('off')
            
            plt.subplot(1, 2, 2)
            plt.imshow(self.output_image)
            plt.title('Manual Colorization', fontsize=14)
            plt.axis('off')
            
            plt.tight_layout()
            if not os.path.exists('outputs'):
                os.makedirs('outputs')
            plt.savefig('outputs/colorization_comparison.png', dpi=300, bbox_inches='tight')
            plt.close()
        except Exception as e:
            print(f"Could not save visualization: {e}")

    def save_image(self):
        if not self.output_image:
            messagebox.showerror("Error", "No colorized image to save!")
            return
            
        save_path = filedialog.asksaveasfilename(
            title="Save Colorized Image",
            defaultextension=".png",
            filetypes=[("PNG files", "*.png"), ("JPEG files", "*.jpg"), ("All files", "*.*")]
        )
        if save_path:
            # Save at original resolution
            high_res_output = self.output_image.resize((1024, 1024), Image.BICUBIC)
            high_res_output.save(save_path, quality=95)
            messagebox.showinfo("Success", f"Image saved to {save_path}")
            self.status_label.config(text=f"Saved: {os.path.basename(save_path)}")

# Main function
def main():
    root = tk.Tk()
    root.geometry("900x600")
    root.resizable(True, True)
    app = ManualColorizationGUI(root)
    
    # Instructions
    instructions = """
    Instructions:
    1. Upload a grayscale image
    2. Draw rectangles on areas you want to colorize
    3. Choose colors for each region
    4. Click 'Apply Colors' to see the result
    5. Use 'Toggle Blend Mode' for different color application styles
    6. Save your colorized image
    """
    
    info_label = tk.Label(root, text=instructions, justify=tk.LEFT, 
                         font=('Arial', 9), bg='lightyellow', relief='ridge', bd=1)
    info_label.pack(side=tk.BOTTOM, fill=tk.X, padx=10, pady=5)
    
    root.mainloop()

if __name__ == "__main__":
    main()
